# Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from scipy.fftpack import dct
from PIL import Image

# DCT Feature Extraction

In [ ]:
def applyDCT(image_path):
    image = load_img(image_path)
    image_gray = image.convert('L') #converted to gray to keep 1 channel = 2D array
    image_array = img_to_array(image_gray) #outputs float32
    dct_transformed = dct(dct(image_array, axis=0, norm='ortho', n=300), axis=1, norm='ortho', n=300) #dct over both axis
    feature_vector = dct_transformed[:45, :45].flatten()
    image.close()
    return feature_vector #shape is (2025,)

In [ ]:
training_dir = r'/content/drive/My Drive/training preprocessed'
# directory with /6 participants/train or test/50 sentences/sample folders/word folders/png images of stacks

In [ ]:
image_paths = []
labels = []
sentence_features = []

participants = os.listdir(training_dir)
participants.sort()
folders = [participant for participant in participants]
for participant in folders:
    print(participant)
    train_path = os.path.join(training_dir, participant, 'train') # change to 'test' when needed
    sentence_folders = os.listdir(train_path)
    sentence_folders.sort()
    for sentence_folder in sentence_folders:
        print(sentence_folder)
        sentence_path = os.path.join(train_path, sentence_folder)
        folders = os.listdir(sentence_path)
        folders.sort()
        for folder in folders:
            folder_path = os.path.join(sentence_path, folder)
            current_sequence = []
            current_labels = []
            folder_paths = os.listdir(folder_path)
            folder_paths.sort()
            for word_folder in folder_paths:
                word_folder_path = os.path.join(folder_path, word_folder)
                words = os.listdir(word_folder_path)
                words.sort()
                for word in words:
                    if word.endswith(".png"):
                        image_path = os.path.join(word_folder_path, word)
                        label = int(os.path.splitext(word)[0])
                        features = applyDCT(image_path)
                        current_sequence.append(features)
                        current_labels.append(label)
            # pad to be 5 words
            if len(current_sequence) < 5:
              last_FV = current_sequence[-1]
              last_label = current_labels[-1]
              for i in range(5 - len(current_sequence)):
                current_sequence.append(last_FV)
                current_labels.append(last_label)

            sentence_features.append(current_sequence)
            labels.append(current_labels)


In [ ]:
np.save(r'/content/drive/My Drive/Model Files/X_test_300_dct.npy', np.array(sentence_features))
np.save(r'/content/drive/My Drive/Model Files/y_test_300_dct.npy', np.array(labels))

In [ ]:
print(len(sentence_features))
print(len(labels))
print(np.array(sentence_features).shape) # (no. samples, 5, 2025)